In [4]:
# Transformer多头attention维度观察
import torch
import torch.nn as nn
torch.manual_seed(224)

In [17]:
query = torch.randint(1,9,(2,8))
query

tensor([[8, 7, 2, 5, 6, 7, 6, 6],
        [2, 2, 3, 8, 6, 7, 6, 7]])

In [18]:
key = torch.randint(1,9,(2,8))
key

tensor([[6, 4, 2, 7, 1, 8, 4, 8],
        [4, 6, 7, 8, 6, 2, 5, 6]])

In [19]:
batch=2
d_model=8
head=4
d_k = 8 // head
print ("batch: ", batch, "d_model: ", d_model, "head: ", head, "d_k: ", d_k)

batch:  2 d_model:  8 head:  4 d_k:  2


In [20]:
query = query.view(2,-1,head,d_k)
key = key.view(2,-1,head,d_k)
print (query)
print (key)

tensor([[[[8, 7],
          [2, 5],
          [6, 7],
          [6, 6]]],


        [[[2, 2],
          [3, 8],
          [6, 7],
          [6, 7]]]])
tensor([[[[6, 4],
          [2, 7],
          [1, 8],
          [4, 8]]],


        [[[4, 6],
          [7, 8],
          [6, 2],
          [5, 6]]]])


In [21]:
query.shape

torch.Size([2, 1, 4, 2])

In [22]:
key.shape

torch.Size([2, 1, 4, 2])

In [23]:
key_ = key.transpose(-2,-1)
key_.shape

torch.Size([2, 1, 2, 4])

In [27]:
key_

tensor([[[[6, 2, 1, 4],
          [4, 7, 8, 8]]],


        [[[4, 7, 6, 5],
          [6, 8, 2, 6]]]])

In [25]:
score = torch.matmul(query,key_)
score

tensor([[[[76, 65, 64, 88],
          [32, 39, 42, 48],
          [64, 61, 62, 80],
          [60, 54, 54, 72]]],


        [[[20, 30, 16, 22],
          [60, 85, 34, 63],
          [66, 98, 50, 72],
          [66, 98, 50, 72]]]])

In [26]:
score.shape

torch.Size([2, 1, 4, 4])

In [43]:
import math
scores = score // math.sqrt(d_k)
print (scores)
print (scores.shape)

tensor([[[[76, 65, 64, 88],
          [32, 39, 42, 48],
          [64, 61, 62, 80],
          [60, 54, 54, 72]]],


        [[[20, 30, 16, 22],
          [60, 85, 34, 63],
          [66, 98, 50, 72],
          [66, 98, 50, 72]]]])
torch.Size([2, 1, 4, 4])


In [44]:
value = torch.randint(1,9,(2,8))
value = value.view(2,-1,head,d_k)
value

tensor([[[[3, 4],
          [6, 3],
          [5, 8],
          [2, 5]]],


        [[[5, 8],
          [7, 1],
          [8, 2],
          [3, 2]]]])

In [45]:
value.shape

torch.Size([2, 1, 4, 2])

In [51]:
softmax = nn.Softmax(dim=-1)
scores = scores.float()
p_attn = softmax(scores)
print (p_attn)
print (p_attn.shape)

tensor([[[[6.1442e-06, 1.0262e-10, 3.7751e-11, 9.9999e-01],
          [1.1224e-07, 1.2309e-04, 2.4723e-03, 9.9740e-01],
          [1.1254e-07, 5.6028e-09, 1.5230e-08, 1.0000e+00],
          [6.1442e-06, 1.5230e-08, 1.5230e-08, 9.9999e-01]]],


        [[[4.5383e-05, 9.9962e-01, 8.3121e-07, 3.3533e-04],
          [1.3888e-11, 1.0000e+00, 7.0955e-23, 2.7895e-10],
          [1.2664e-14, 1.0000e+00, 1.4252e-21, 5.1091e-12],
          [1.2664e-14, 1.0000e+00, 1.4252e-21, 5.1091e-12]]]])
torch.Size([2, 1, 4, 4])


In [53]:
output = torch.matmul(p_attn, value.float()) # [batch_size,1,4,4] matmul [batch_size,1,4,2] --- [batch_size,1,4,2]
output.shape

torch.Size([2, 1, 4, 2])

In [60]:
x = output.transpose(1, 2).view(batch, -1, head * d_k)

In [61]:
x.shape

torch.Size([2, 1, 8])

原代码地址:[Pytorch-Attention](https://github.com/harvardnlp/annotated-transformer/blob/master/The%20Annotated%20Transformer.ipynb)
1. multi_head体现在对特征维度上进行reshape(batch,-1,head,d_k)得到若干个子空间，分别在子空间上计算attention
2. 多头计算后的concat同样通过view进行，例如Line60
3. 注意torch.matmul的用法